# KBO 경기정보 크롤러
https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx

In [54]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 n

In [56]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import random
import pandas as pd
import os

## KBO 타자 기록실 연결

In [57]:
def connect_KBO():
    global driver
    global act

    # 드라이버 연결
    driver = webdriver.Chrome()

    # 액션체인 객체 생성
    act = ActionChains(driver)

    # KBO 기록실 이동 - 타자 / 투수
    driver.get("https://www.koreabaseball.com/Schedule/Schedule.aspx")

    # 대기
    driver.implicitly_wait(3)

## 연도별 경기정보 수집기

In [58]:
def get_games_info(year):
    games_info = list()

    year_btn = driver.find_element(By.XPATH, '//*[@id="ddlYear"]')
    sel_year_btn = Select(year_btn)
    sel_year_btn.select_by_value(str(year))
    driver.implicitly_wait(random.randrange(3, 7))

    for month in range(1, 13):
        month_btn = driver.find_element(By.XPATH, '//*[@id="ddlMonth"]')
        sel_month_btn = Select(month_btn)

        month = "0"+str(month) if month < 10 else str(month)
        sel_month_btn.select_by_value(month)
        driver.implicitly_wait(random.randrange(3, 7))

        table = driver.find_element(By.XPATH, '/html/body/form/div[3]/section/div/div/div[4]/table/tbody')
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 데이터가 없는 월은 continue
        if len(rows) == 1:
            continue


        game_date = None

        for tr in rows:
            info = list()
            datas = tr.find_elements(By.TAG_NAME, "td")

            for td in datas:
                if td.get_attribute("rowspan"):
                    game_date = str(year) + "-" + td.text[:5].replace(".", "-")
                if ":" in td.text:
                    game_start_time = td.text
                if "vs" in td.text:
                    away_name, home_name = td.text.split("vs")
                    away_name = "".join([i for i in away_name if i.isnumeric() == False])
                    home_name = "".join([i for i in home_name if i.isnumeric() == False])

                if td == datas[-2]:
                    game_place = td.text

            games_info.append([game_date, game_place, home_name, away_name, game_start_time])

    return games_info

## 크롤러 실행기

In [59]:
def runner(year):
    # 필요한 데이터 피쳐 리스트
    col = ["game_date", "game_place", "home_name", "away_name", "game_start_time"]
    f_name = f"KBO경기정보_{year}.csv"
    print(f"현재 경로: [{os.getcwd()}]에 [{f_name}] 로 저장예정. \n원치 않으면 실행을 중단하세요.")

    connect_KBO()
    data = get_games_info(year)

    df = pd.DataFrame(data, columns=col)
    df.to_csv(f_name, index=False)

    print(f"파일 저장 완료\n파일 저장 경로: {os.getcwd()}/{f_name}")

    return df

In [60]:
runner(2020)

현재 경로: [/content]에 [KBO경기정보_2020.csv] 로 저장예정. 
원치 않으면 실행을 중단하세요.


WebDriverException: ignored